# Spatial Lag - Random Effects Panel Model

This notebook introduces the Spatial Lag model for Random Effects Panel data. It is based on the estimation procedure outline in:
- Anselin, Le Gallo and Jayet (2008). Spatial Panel Econometrics.
- Elhorst (2014). Spatial Econometrics, From Cross-Sectional Data to Spatial Panels.

In [1]:
import libpysal
import spreg
import numpy as np
import numpy.linalg as la
from scipy import sparse as sp
from scipy.sparse.linalg import splu as SuperLU
from spreg.utils import RegressionPropsY, RegressionPropsVM, inverse_prod, set_warn
from spreg.sputils import spdot, spfill_diagonal, spinv
import spreg.diagnostics as DIAG
import spreg.user_output as USER
import spreg.summary_output as SUMMARY
try:
    from scipy.optimize import minimize_scalar
    minimize_scalar_available = True
except ImportError:
    minimize_scalar_available = False
    
from spreg.panel_utils import check_panel, demean_panel

### Read data

In [2]:
from libpysal.weights import w_subset
import pandas as pd
# Open data on NCOVR US County Homicides (3085 areas).
nat = libpysal.examples.load_example("NCOVR")
db = libpysal.io.open(nat.get_path("NAT.dbf"), "r")
# Create spatial weight matrix
nat_shp = libpysal.examples.get_path("NAT.shp")
w_full = libpysal.weights.Queen.from_shapefile(nat_shp)

# Define dependent variable
name_y = ["HR70", "HR80", "HR90"]
y_full = np.array([db.by_col(name) for name in name_y]).T
# Define independent variables
name_x = ["RD70", "RD80", "RD90", "PS70", "PS80", "PS90"]
x_full = np.array([db.by_col(name) for name in name_x]).T

epsilon = 0.0000001

We will work with a subset of the data, to get a faster implementation of the random effects estimation.

In [3]:
name_c = ["STATE_NAME", "FIPSNO"]
df_counties = pd.DataFrame([db.by_col(name) for name in name_c], index=name_c).T

filter_states = ["Arkansas", "Kansas", "Missouri", "Oklahoma"]
filter_counties = df_counties[df_counties["STATE_NAME"].isin(filter_states)]["FIPSNO"].values

counties = np.array(db.by_col("FIPSNO"))
subid = np.where(np.isin(counties, filter_counties))[0]

w = w_subset(w_full, subid)
w.transform = 'r'

y = y_full[subid, ]
x = x_full[subid, ]

### Transform variables

In [4]:
# Check the data structure and converts from wide to long if needed.
bigy, bigx, name_y, name_x = check_panel(y, x, w, name_y, name_x)

Similarly, assuming x[:, 0:T] refers to T periods of k1, x[:, T+1:2T] refers to k2, etc.


In [5]:
# In random effects, we need can include the constant
name_x = ["constant", "RD", "PS"]
ones = np.ones((bigx.shape[0], 1))
bigx = np.hstack((ones, bigx))

In [6]:
n = w.n
t = bigy.shape[0] // n
k = bigx.shape[1]
# Big W matrix
W = w.full()[0]
W_nt = np.kron(np.identity(t), W)
Wsp = w.sparse
Wsp_nt = sp.kron(sp.identity(t), Wsp, format="csr")

### Estimation

First, we need to maximize the concentrated log-likehood function with respect to $\phi$, assuming an initial value of $\beta$ and $\rho$:
$$
L = - \frac{NT}{2} \ln (e'_p e_p) + \frac{N}{2} \ln \phi^2
$$

where $e_p = \tilde{y} - \rho W \tilde{y} - \tilde{X} \beta$ and $\tilde{y} = Q_{\phi} y$.

Also, $Q_{\phi}$ is defined as:
$$
Q_{\phi} = \left[ I_T - \phi \left( \iota \cdot \iota' / t \right) \right] \otimes I_N
$$

where $0 \leq \phi^2 = \frac{\sigma^2}{T \sigma^2_u + \sigma^2} \leq 1$

Then, maximize the concentrated log-likehood function with respect to $\rho$, as we do with the fixed effects estimation:
$$
L = \frac{NT}{2} \ln (e'_r e_r) - T \ln | I_N - \rho W |
$$

where $e_r = e_0 - \rho e_1$. 

In [7]:
def lag_c_loglik_sp(rho, n, t, e0, e1, I, Wsp):
    # concentrated log-lik for lag model, sparse algebra
    if isinstance(rho, np.ndarray):
        if rho.shape == (1, 1):
            rho = rho[0][0]
    er = e0 - rho * e1
    sig2 = spdot(er.T, er) / (n*t)
    nlsig2 = (n*t / 2.0) * np.log(sig2)
    a = I - rho * Wsp
    LU = SuperLU(a.tocsc())
    jacob = t * np.sum(np.log(np.abs(LU.U.diagonal())))
    clike = nlsig2 - jacob
    return clike

In [8]:
def phi_c_loglik_sp(phi, rho, beta, bigy, bigx, n, t, Wsp_nt):
    # Demeaned variables
    y = demean_panel(bigy, n, t, phi=phi)
    x = demean_panel(bigx, n, t, phi=phi)
    # Lag dependent variable
    ylag = spdot(Wsp_nt, y)
    er = y - rho*ylag - spdot(x, beta)
    sig2 = spdot(er.T, er)
    nlsig2 = (n*t / 2.0) * np.log(sig2)
    nphi2 = (n / 2.0) * np.log(phi**2)
    clike = nlsig2 - nphi2
    return clike

In [9]:
converge = 1
criteria = 0.0000001
i = 0
itermax = 100
I = sp.identity(n)
rho = 0.1
xtx = spdot(bigx.T, bigx)
xtxi = la.inv(xtx)
xty = spdot(bigx.T, bigy)
b = spdot(xtxi, xty)
phi = 0.1

while converge > criteria and i < itermax:
    phiold = phi
    res_phi = minimize_scalar(phi_c_loglik_sp, 0.1, bounds=(0.0, 1.0),
                           args=(rho, b, bigy, bigx, n, t, Wsp_nt), 
                           method='bounded', options={"xatol": epsilon})
    phi = res_phi.x[0][0]
    # Demeaned variables
    y = demean_panel(bigy, n, t, phi=phi)
    x = demean_panel(bigx, n, t, phi=phi)
    # Lag dependent variable
    ylag = spdot(Wsp_nt, y)
    # b0, b1, e0 and e1
    xtx = spdot(x.T, x)
    xtxi = la.inv(xtx)
    xty = spdot(x.T, y)
    xtyl = spdot(x.T, ylag)
    b0 = spdot(xtxi, xty)
    b1 = spdot(xtxi, xtyl)
    e0 = y - spdot(x, b0)
    e1 = ylag - spdot(x, b1)
    
    res_rho = minimize_scalar(lag_c_loglik_sp, 0.0, bounds=(-1.0, 1.0),
                              args=(n, t, e0, e1, I, Wsp), 
                              method='bounded', options={"xatol": epsilon})
    rho = res_rho.x[0][0]
    # b, residuals and predicted values
    b = b0 - rho * b1
    
    i += 1
    converge = np.abs(phiold - phi)

Calculate betas as:
$$
\beta = \beta_o - \rho \beta_1
$$

In [10]:
# b, residuals and predicted values
betas = np.vstack((b, rho))   # rho added as last coefficient
betas

array([[4.44421994],
       [2.52821717],
       [2.24768846],
       [0.25846846]])

Calculate $\sigma^2$ as:
$$
\sigma^2 = (e_0 - \rho \cdot e_1)' (e_0 - \rho \cdot e_1)
$$

In [11]:
# compute full log-likelihood, including constants
ln2pi = np.log(2.0 * np.pi)
llik = -res_rho.fun - (n*t) / 2.0 * ln2pi - (n*t) / 2.0
logll = llik[0][0]

# Calculate sigma2
u = e0 - rho * e1
sig2 = spdot(u.T, u) / n*t

### Variance matrix

$$
Var[\beta, \delta, \sigma^2] = 
\begin{pmatrix}
\frac{X'X}{\sigma^2}               &                                               &  \\ 
X' (I_T \otimes \tilde{W}) X \beta & T \cdot tr(\tilde{W}^2 + \tilde{W}'\tilde{W}) + \beta' X' (I_T \otimes \tilde{W}'\tilde{W}) X \beta &  \\ 
0                                  & \frac{T}{\sigma^2} tr(\tilde{W}) & \frac{NT}{2 \sigma^4} \\
\end{pmatrix}
$$

where $\tilde{W} = W (I_N - \rho W)^{-1}$

In [12]:
predy = y - u
xb = spdot(x, b)
predy_e = inverse_prod(
    Wsp_nt, xb, rho, inv_method="power_exp", threshold=epsilon)
e_pred = y - predy_e

In [13]:
# information matrix
a = -rho * W
spfill_diagonal(a, 1.0)
ai = spinv(a)
wai = spdot(Wsp, ai)
tr1 = wai.diagonal().sum() #same for sparse and dense

wai2 = spdot(wai, wai)
tr2 = wai2.diagonal().sum()

waiTwai = spdot(wai.T, wai)
tr3 = waiTwai.diagonal().sum()

wai_nt = sp.kron(sp.identity(t), wai, format="csr")
wpredy = spdot(wai_nt, xb)
xTwpy = spdot(x.T, wpredy)

waiTwai_nt = sp.kron(sp.identity(t), waiTwai, format="csr")
wTwpredy = spdot(waiTwai_nt, xb)
wpyTwpy = spdot(xb.T, wTwpredy)

# order of variables is beta, rho, sigma2
v1 = np.vstack(
    (xtx/sig2, xTwpy.T/sig2, np.zeros((2, k))))
v2 = np.vstack(
    (xTwpy/sig2, t*(tr2+tr3) + wpyTwpy/sig2, -tr1/sig2, t*tr1/sig2))
v3 = np.vstack(
    (np.zeros((k, 1)), -tr1/sig2, n*(t + 1/phi**2), -n/sig2))
v4 = np.vstack(
    (np.zeros((k, 1)), t*tr1/sig2, -n/sig2**2, n*t/(2.0*sig2**2)))

v = np.hstack((v1, v2, v3, v4))

vm1 = la.inv(v)  # vm1 includes variance for sigma2
vm = vm1[:-2, :-2]  # vm is for coefficients only
vm

array([[ 0.37323335,  0.0067311 ,  0.15857564, -0.01069964],
       [ 0.0067311 ,  0.39643298,  0.01546323, -0.004546  ],
       [ 0.15857564,  0.01546323,  0.48536089, -0.00324211],
       [-0.01069964, -0.004546  , -0.00324211,  0.00189414]])

# R section

In [1]:
### set options
options(prompt = "R> ",  continue = "+ ", width = 70, useFancyQuotes = FALSE, warn=-1)

### load library
library("splm")

Loading required package: spdep

Loading required package: sp

Loading required package: spData

To access larger datasets in this package, install the
spDataLarge package with: `install.packages('spDataLarge',
repos='https://nowosad.github.io/drat/', type='source')`

Loading required package: sf

Linking to GEOS 3.8.0, GDAL 3.0.4, PROJ 6.3.1



In [2]:
## read data
nat <- read.csv("data/sub_NAT.csv", header = TRUE)
wnat <- as.matrix(read.csv("data/sub_NAT_w.csv", header = FALSE))
## standardization
wnat <- wnat/apply(wnat, 1, sum)
## make it a listw
lwnat <- mat2listw(wnat)

col_order <- c("FIPSNO", "YEAR", "HR", "RD", "PS")
nat <- nat[, col_order]

In [3]:
fixed_lag = spml(HR ~ RD + PS, data=nat, listw=lwnat, effect="individual",
                 model="random", spatial.error = "none", lag=TRUE)

In [4]:
summary(fixed_lag)

ML panel with spatial lag, random effects 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl)

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -8.379  -1.809   0.278   1.349   3.375  39.269 

Error variance parameters:
    Estimate Std. Error t-value  Pr(>|t|)    
phi 0.378582   0.064757  5.8462 5.029e-09 ***

Spatial autoregressive coefficient:
       Estimate Std. Error t-value  Pr(>|t|)    
lambda 0.258468   0.038933  6.6389 3.161e-11 ***

Coefficients:
            Estimate Std. Error t-value  Pr(>|t|)    
(Intercept)  4.44422    0.18643 23.8390 < 2.2e-16 ***
RD           2.52822    0.20697 12.2155 < 2.2e-16 ***
PS           2.24769    0.23089  9.7347 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [35]:
betas

array([[4.44421992],
       [2.52821676],
       [2.24768837],
       [0.25846847]])

In [18]:
1/(t*phi**2) - 1/t

0.37858278205185997

In [5]:
fixed_lag$vcov

,(Intercept),RD,PS
(Intercept),0.034754804,-0.0021053688,0.0155846231
RD,-0.002105369,0.0428358261,0.0008535634
PS,0.015584623,0.0008535634,0.0533123892


In [6]:
fixed_lag$vcov.arcoef

,lambda
lambda,0.003841403


In [33]:
np.set_printoptions(suppress=True)
np.around(vm, decimals=8)

array([[ 0.37323359,  0.00673109,  0.15857575, -0.01069964],
       [ 0.00673109,  0.39643321,  0.01546323, -0.004546  ],
       [ 0.15857575,  0.01546323,  0.48536125, -0.00324211],
       [-0.01069964, -0.004546  , -0.00324211,  0.00189414]])

In [6]:
fixed_lag = spml(HR ~ RD + PS, data=nat, listw=lwnat, effect="individual",
                 model="random", spatial.error = "b", lag=FALSE)

In [7]:
summary(fixed_lag)

ML panel with , random effects, spatial error correlation 

Call:
spreml(formula = formula, data = data, index = index, w = listw2mat(listw), 
    w2 = listw2mat(listw2), lag = lag, errors = errors, cl = cl)

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-10.940  -3.157  -0.869  -0.012   2.147  36.150 

Error variance parameters:
    Estimate Std. Error t-value  Pr(>|t|)    
phi 0.304972   0.060005  5.0825 3.725e-07 ***
rho 0.347149   0.047581  7.2960 2.964e-13 ***

Coefficients:
            Estimate Std. Error t-value  Pr(>|t|)    
(Intercept)  5.87150    0.22920  25.617 < 2.2e-16 ***
RD           3.22219    0.23425  13.755 < 2.2e-16 ***
PS           2.60396    0.24820  10.491 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
